# BigKinds에서 제공한 엑셀파일 DB에 넣는 코드

In [1]:
import pandas as pd
import glob
import os
import psycopg2

In [2]:
use_col = ['일자','언론사','본문','URL']
col_rename = ['date','publisher','content','url']

In [3]:

# Excel 파일이 있는 디렉토리 경로
folder_path = "/Users/kim-youngho/git/GeoNewsApt/data/news/bigkinds"

# 폴더 내 모든 .xlsx 파일 경로 리스트업
xlsx_files = [os.path.join(folder_path, file)
              for file in os.listdir(folder_path)
              if file.endswith('.xlsx')]

In [4]:
# 파일들을 DataFrame으로 읽어서 리스트에 저장
df_list = [pd.read_excel(file) for file in xlsx_files]

/Users/kim-youngho/apt_price/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/kim-youngho/apt_price/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/kim-youngho/apt_price/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/kim-youngho/apt_price/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/kim-youngho/apt_price/lib/python3.11/site

In [5]:
# 하나의 DataFrame으로 병합
df = pd.concat(df_list, ignore_index=True)

In [6]:
df = df[use_col]

In [7]:
df.columns = col_rename

In [8]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

In [9]:
df.head()

,date,publisher,content,url
0,2025-08-19,한국경제,이 기사는 08월 19일 14:23 마켓인사이트에 게재된 기사입니다. \n \n \...,https://www.hankyung.com/article/202508174248r
1,2025-08-19,한국경제,이 기사는 08월 19일 11:31 마켓인사이트에 게재된 기사입니다. \n \n \...,https://www.hankyung.com/article/202508197898r
2,2025-08-19,한국경제,이 기사는 08월 19일 17:14 마켓인사이트에 게재된 기사입니다. \n \n \...,https://www.hankyung.com/article/202508198739r
3,2025-08-19,한국경제,이 기사는 08월 19일 15:42 마켓인사이트에 게재된 기사입니다. \n \n \...,https://www.hankyung.com/article/202508186828r
4,2025-08-19,한국경제,이 기사는 08월 19일 11:38 마켓인사이트에 게재된 기사입니다. \n \n \...,https://www.hankyung.com/article/202508185747r


In [10]:
df.date.unique()

<DatetimeArray>
['2025-08-19 00:00:00', '2025-08-18 00:00:00', '2025-08-17 00:00:00',
 '2025-08-16 00:00:00', '2025-08-15 00:00:00', '2025-08-14 00:00:00',
 '2025-08-13 00:00:00', '2025-08-12 00:00:00', '2025-08-11 00:00:00',
 '2025-08-10 00:00:00',
 ...
 '2020-07-19 00:00:00', '2020-07-18 00:00:00', '2020-07-17 00:00:00',
 '2020-07-16 00:00:00', '2020-07-15 00:00:00', '2020-07-14 00:00:00',
 '2020-07-13 00:00:00', '2020-07-12 00:00:00', '2020-07-11 00:00:00',
 '2020-07-10 00:00:00']
Length: 1835, dtype: datetime64[ns]

In [11]:
df[df.publisher == '한국경제'].content[0]

'이 기사는 08월 19일 14:23 마켓인사이트에 게재된 기사입니다. \n \n \n \nNH투자증권이 올 상반기 국내 주요 증권사 중 투자은행(IB) 사업 실적을 가장 가파르게 끌어올렸다. 정통 IB 사업 강화를 천명한 메리츠증권도 반기 IB 순영업수익 2000억원 문턱을 넘어서며 전통의 강호 한국투자증권과 KB증권을 위협하는 존재로 성장했다. 반면 미래에셋..'

## 디비 접속

In [12]:

conn = psycopg2.connect(
    host="127.0.0.1",
    port=5432,
    user="postgres",
    password="2464",
    dbname="GeoNewsApt"
)
cur = conn.cursor()

In [13]:
df.columns

Index(['date', 'publisher', 'content', 'url'], dtype='object')

In [14]:
insert_sql = """
INSERT INTO news (date, publisher, content, url) 
VALUES (%s, %s, %s, %s)
"""

In [15]:
for _, row in df.iterrows():
    date = row['date']
    publisher = row['publisher']
    content = row['content']
    url = row['url']
    cur.execute(insert_sql, (date, publisher, content, url))

conn.commit()
cur.close()
conn.close()